# Long term outcomes in pediatric patients undergoing ventriculostomy


In [1]:
import sys
sys.path.append('../')
import chlib
from chlib.entity import aggregate
from chlib.codes import Coder
from pprint import pprint 
from collections import defaultdict
reload(chlib)
json_config ='../config.json'
HCUPSID = chlib.data.Data.get_from_config(json_config,'HCUPSID')
HCUPSID.load_codes()
code = 'P022'
P022 = HCUPSID.get_code('P022')
coder = Coder()
print coder[code]
print P022

Ventriculostomy
Code: P022 in Dataset: HCUP CA, FL & NY SID 2006-2011
visits 44678
By years:
[ (2006, 7703),
  (2007, 7712),
  (2008, 7942),
  (2009, 7764),
  (2010, 7784),
  (2011, 5773)]
By states:
[('NY', 12383), ('CA', 20375), ('FL', 11920)]
By visit type:
[(0, 44678)]


### Retrieve all patients who underwent the procedure

In [2]:
patients = [p for _,p in HCUPSID.iter_patients_by_code('P022') if p.linked]
print "Number of patients {}".format(len(patients))

Number of patients 36378


### Set up selection criteria Age < 19 years and underwent Ventriculostomy in 2006

In [3]:
def criteria(p):
    if p.visits[0].age < 19: # the patient is younger than 19 years during first visit recorded for the patient 
        for v in p.visits: # iterate over all visits
            for pr in v.prs: # iterate over all procedures
                if pr.pcode == code and v.year == 2006: # if there is a procedure during 2006 return True
                    return True
    return False
P022_in_2006 = filter(criteria,patients) # filter list of patients using above criteria
print "Number of patients who underwent {} during 2006, at age < 19 or younger is {}".format(code,len(P022_in_2006))

Number of patients who underwent P022 during 2006, at age < 19 or younger is 754


### Out of those patients how may appear in subsequent years and how often are they hospitalized

In [4]:
patients_years = defaultdict(set)
visits_years = defaultdict(int)
for p in P022_in_2006:
    for v in p.visits:
        patients_years[v.year].add(p.patient_key)
        visits_years[v.year] += 1
            
print "year","patients","\tinpatient visits"
for k in sorted([(k,len(v),visits_years[k]) for k,v in patients_years.iteritems()]):
    print "{}\t{}\t  {}".format(k[0],k[1],k[2])

year patients 	inpatient visits
2006	754	  1907
2007	256	  674
2008	144	  299
2009	125	  242
2010	104	  198
2011	85	  172


### Filtering further to select only patients/visits in 2011 ( approximately > 4 years )

In [5]:
def criteria_2011(p):
    for v in p.visits:
        if v.year == 2011:
            return True
    return False

def remove_visits_before_2011(p):
    temp = [v for v in p.visits if v.year == 2011]
    del p.visits[:]
    p.visits.extend(temp)
    return p

P022_in_2006_and_2011 =[remove_visits_before_2011(p) for p in filter(criteria_2011,P022_in_2006)]

### Compute Aggregate Statistics on those patients

In [6]:
pagg = aggregate.PAggregate()
pagg.init_compute("temp","temp",aggregate.Policy(min_count=20,min_hospital=2,min_subset=20,base=10)) 
for p in P022_in_2006_and_2011:
    pagg.add_patient(p)
results_available = pagg.end_compute()

### Primary and Secondary Diagnoses during 2011

In [7]:
for s in pagg.obj.subsets:
    for dx in s.subset.dxh:
        print u"{}\t{}\t{}".format(dx.primary,dx.all,coder[dx.k])

0	40	Obstructive hydrocephalus
0	20	Infantile cerebral palsy, unspecified
0	60	Presence of cerebrospinal fluid drainage device
0	50	Epilepsy, unspecified, without mention of intractable epilepsy
0	30	Gastrostomy status
0	20	Asthma, unspecified type, unspecified


### Inpatient procedures during 2011

In [8]:
for s in pagg.obj.subsets:
    for pr in s.subset.prh:
        print "{}\t{}".format(pr.v,coder[pr.k])